In [1]:
!git clone https://github.com/richardyy1188/Pretrain-MLM-and-finetune-on-GLUE-with-fastai.git
%cd Pretrain-MLM-and-finetune-on-GLUE-with-fastai
%pip install -q fastai2 transformers h5py

Cloning into 'Pretrain-MLM-and-finetune-on-GLUE-with-fastai'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 24 (delta 6), reused 20 (delta 5), pack-reused 0
Unpacking objects: 100% (24/24), done.
/content/Pretrain-MLM-and-finetune-on-GLUE-with-fastai
     |████████████████████████████████| 194kB 2.7MB/s 
     |████████████████████████████████| 645kB 8.5MB/s 
     |████████████████████████████████| 3.8MB 13.9MB/s 
     |████████████████████████████████| 1.1MB 32.2MB/s 
     |████████████████████████████████| 890kB 21.3MB/s 


In [2]:
from IPython.core.debugger import set_trace as bk
from functools import partial
import h5py
import torch
from fastai2.text.all import *
from transformers import ElectraTokenizer
hf_tokenizer = ElectraTokenizer.from_pretrained("google/electra-small-generator")
from _utils.hf_transformers_integration import HF_Tokenizer, HF_TextBlock, HFModelWrapper

In [3]:
# from the lyrics of "Avicii - Waiting For Love"
df = pd.DataFrame(data={'text':["Monday left me broken","Tuesday I was through with hoping","Wednesday my empty arms were open","Thursday waiting for love, waiting for love","Thank the stars it's Friday","I'm burning like a fire gone wild on Saturday","Guess I won't be coming to church on Sunday","I'll be waiting for love, waiting for love","To come around",],'is_valid':[False]*7 + [True]*2})
df

,text,is_valid
0,Monday left me broken,False
1,Tuesday I was through with hoping,False
2,Wednesday my empty arms were open,False
3,"Thursday waiting for love, waiting for love",False
4,Thank the stars it's Friday,False
5,I'm burning like a fire gone wild on Saturday,False
6,Guess I won't be coming to church on Sunday,False
7,"I'll be waiting for love, waiting for love",True
8,To come around,True


#1. TextDataloader

In [0]:
@delegates()
class TextDataloader(TfmdDL):
  def __init__(self, dataset, max_seq_len=float('inf'), sort_by_len=True, agg_mode=None, ignore_gt_maxlen=True, remove_heads=False, remove_tails=False, bos_idx_add=None, eos_idx_add=None, **kwargs):
    super().__init__(dataset, **kwargs)
    assert agg_mode in [None, 'lm', 'lines', 'window']
    assert not (agg_mode and max_seq_len is None) 
    self.sort_by_len = sort_by_len and agg_mode in [None, 'lines'] # sorting makes sense only with these modes
    ignore_gt_maxlen = ignore_gt_maxlen and agg_mode in [None, 'lines'] and max_seq_len is not None
    first_text_tensor = dataset[0][0]
    device, dtype = first_text_tensor.device, first_text_tensor.dtype
    self.bos = torch.tensor([bos_idx_add] if bos_idx_add is not None else [], device=device, dtype=dtype)
    self.eos = torch.tensor([eos_idx_add] if eos_idx_add is not None else [], device=device, dtype=dtype)

    store_attr(self,'dataset,max_seq_len,sort_by_len,agg_mode,ignore_gt_maxlen,remove_heads,remove_tails')
    
    self.samples = L()
    # residual_len will reset to initial_residual_len
    # lm mode: max_seq_len text and 1 right-shift text, so take max_seq_len + 1 window
    self.initial_residual_len = max_seq_len + 1 if agg_mode=='lm' else max_seq_len 
    # keep spaces to add bos to final text 
    if bos_idx_add is not None: self.initial_residual_len -= 1
    if eos_idx_add is not None: self.initial_residual_len -= 1
    self.residual_len, self.new_sample = self.initial_residual_len, []
    # only use [start:end] text to concatenate (if needed)
    self.start = 0 if not remove_heads else 1
    self.end = None if not remove_tails else -1

    for i, sample in enumerate(dataset):
      line_len = len(sample[0])
      if remove_heads: line_len -= 1
      if remove_tails: line_len -= 1
      
      if max_seq_len is not None and line_len > self.initial_residual_len and agg_mode in [None, 'lines']:
        if ignore_gt_maxlen: continue
        else: raise ValueError(f'The {i} th text line in dataset has length {line_len}(without removing head or tail, {len(sample[0])}), and is longer than max length {self.initial_residual_len}(without add bos or eos, {max_seq_len})')
        
      if agg_mode is None: self.samples.append( (line_len, i) )
      elif agg_mode == 'lines': self._accumulate_lines(i, line_len)
      else: self._accumulate_window(i, line_len)

    # specify total number of samples
    self.n = len(self.samples)
      
  def _accumulate_lines(self, i, line_len):
    if line_len <= self.residual_len:
      self.new_sample.append(i)
      self.residual_len -= line_len
    else:
      self.samples.append((self.max_seq_len-self.residual_len, self.new_sample))
      self.new_sample = [i]
      self.residual_len = self.initial_residual_len - line_len

  def _accumulate_window(self, i, line_len):
    usable_len = line_len
    cursor = self.start
    while usable_len != 0:
      use_len = min(usable_len, self.residual_len)
      self.new_sample.append((i, cursor, cursor+use_len))
      self.residual_len -= use_len
      usable_len -= use_len
      cursor += use_len
      if self.residual_len == 0:
        self.samples.append(self.new_sample)
        self.new_sample = []
        self.residual_len = self.initial_residual_len

  def create_item(self, s):
    if self.agg_mode is None:
      "samples = [ (length, idx), ... ]"
      idx = self.samples[s][1]
      sample = self.dataset[idx]
      text= TensorText(torch.cat([self.bos, sample[0][self.start:self.end], self.eos]))
      return ( text, *sample[1:] )
    elif self.agg_mode == 'lines':
      "samples = [ (length, [idx, idx, ...]) , ... ]"
      agg_text = concat(*[ self.dataset[idx][0][self.start:self.end] for idx in self.samples[s][1] ] )
      agg_text = TensorText(torch.cat([self.bos, agg_text, self.eos]))
      return (agg_text, )
    else: # window or lm
      "samples = [ (idx,start,end) ]"
      agg_text = concat(*[ self.dataset[idx][0][start:end] for idx,start,end in self.samples[s] ])
      agg_text = TensorText(torch.cat([self.bos, agg_text, self.eos]))
      if self.agg_mode == 'window':
        return (agg_text, )
      else: # 'lm'
        return (LMTensorText(agg_text[:-1]), agg_text[1:])

  def shuffle_fn(self, idxs):
    if self.agg_mode in ['lm', 'window']:
      self.samples.shuffle()
    else:
      self.samples.sort(key=lambda s: s[0])
    return idxs

  @delegates(TfmdDL.new)
  def new(self, dataset=None, **kwargs):
    # assume val data should use the same max_seq_len with train data, single_line
    'dataset,max_seq_len,sort_by_len,agg_mode,concat,ignore_gt_maxlen'
    return TextDataloader(dataset=dataset,
                          # if lm, pass max_seq_len after restore to original value
                          max_seq_len=self.max_seq_len,
                          sort_by_len=False, # valid set even don't shuffle
                          agg_mode=self.agg_mode,
                          ignore_gt_maxlen=False, # You can't discard data from valid set, especially test set
                          **kwargs,
    )

# 2. Valid dataloader problem

`Filteredbase.dataloaders` didn't merge kwargs When creating validation dataloader, 
which makes validation dataloader don't have `pad_input_chunk` as before_batch transform,
that specified in `Textblock.__init__`

In [0]:
def mydataloaders(self, bs=64, val_bs=None, shuffle_train=True, n=None, path='.', dl_type=None, dl_kwargs=None,
                    device=None, **kwargs):
  if device is None: device=default_device()
  if dl_kwargs is None: dl_kwargs = [{}] * self.n_subsets
  if dl_type is None: dl_type = self._dl_type
  drop_last = kwargs.pop('drop_last', shuffle_train)
  dl = dl_type(self.subset(0), bs=bs, shuffle=shuffle_train, drop_last=drop_last, n=n, device=device,
               **merge(kwargs, dl_kwargs[0]))
  dls = [dl] + [dl.new(self.subset(i), bs=(bs if val_bs is None else val_bs), shuffle=False, drop_last=False,
                       n=None, **merge(kwargs, dl_kwargs[i])) for i in range(1, self.n_subsets)]
  return self._dbunch_type(*dls, path=path, device=device)

class MyDataBlock(DataBlock):
  def dataloaders(self, source, path='.', verbose=False, **kwargs):
    dsets = self.datasets(source)
    kwargs = {**self.dls_kwargs, **kwargs, 'verbose': verbose}
    return mydataloaders(dsets, path=path, after_item=self.item_tfms, after_batch=self.batch_tfms, **kwargs)

# 3. Try

We'll try different param of `TextDataloader` to show its capability, but **!! it doesn't mean these are the best practices. !!**

In [0]:
db = DataBlock(splitter=ColSplitter(),
              blocks=HF_TextBlock.from_df('text', hf_tokenizer),
              get_x=ColReader('text'),)

Default behavior:
* a line a sample
* collect samples by their length. (try to make samples with the same length as a batch, to reduce number of pad)

In [7]:
dls = db.dataloaders(df, bs=4, shuffle_train=True, pad_first=False, dl_type=TextDataloader)
dls.show_batch(max_n=4)
"""
We sort the sample by its length.
Observe that the 3rd sample of batch is Friday (9 tokens) but not Thursday (10 tokens), 
thus we can reduce number of pad need to add, 
becuase we have to make all samples in a batch the same legth.
"""
print('x batch size:', dls.one_batch()[0].shape)

,text
0,[CLS] monday left me broken [SEP] [PAD] [PAD] [PAD]
1,[CLS] tuesday i was through with hoping [SEP] [PAD]
2,[CLS] wednesday my empty arms were open [SEP] [PAD]
3,[CLS] thank the stars it ' s friday [SEP]


x batch size: torch.Size([4, 9])


**Window mode**
* Want to use broader context
* sliding context window
* less pad (only samples in the last batch may have pad)
* every sample is of `max_seq_len` length. (Unless the last batch only have one sample shorter than `max_seq_len`)

In [8]:
dls = db.dataloaders(df, shuffle_train=False, pad_first=False, bs=2,
                     dl_type=partial(TextDataloader,
                                     max_seq_len=15,
                                     agg_mode='window',
                                     remove_heads=True,
                                     remove_tails=True,
                                     bos_idx_add=hf_tokenizer.cls_token_id,
                                     eos_idx_add=hf_tokenizer.sep_token_id))
dls.show_batch(max_n=2)
"""
To use CLS...SEP format, first remove heads(CLS) and tails(SEP) for every line,
and then add bos(CLS) and eos(SEP) to the head and tail of concatenated sequence.
"""
print('x batch size:', dls.one_batch()[0].shape)

,text
0,[CLS] monday left me broken tuesday i was through with hoping wednesday my empty [SEP]
1,"[CLS] arms were open thursday waiting for love , waiting for love thank the [SEP]"


x batch size: torch.Size([2, 15])


**Lines mode**
* Want to attend to wider context, but also don't want shattered sentence.
* Sequentially concat lines.
* Note that `max_seq_len` is not definitely length of sample, and increasing it doesn't definitely increase number of pads used.

In [9]:
dls = db.dataloaders(df, shuffle_train=False, pad_first=False, bs=2,
                     dl_type=partial(TextDataloader,
                                     max_seq_len=13,
                                     agg_mode='lines',
                                     remove_heads=True,
                                     bos_idx_add=hf_tokenizer.cls_token_id))
dls.show_batch(max_n=2)
"""
To get CLS ... SEP ... SEP format, we remove head (CLS) for every line,
and add back an bos (CLS) to head of concated sample.
"""
print('x batch size:', dls.one_batch()[0].shape)

,text
0,[CLS] monday left me broken [SEP] tuesday i was through with hoping [SEP]
1,[CLS] wednesday my empty arms were open [SEP] [PAD] [PAD] [PAD] [PAD] [PAD]


x batch size: torch.Size([2, 13])


**(Traditional) Language model mode**
* predict i th token in y, using 0~i-1 tokens in x
* sliding context window
* samples in the last batch may have pad
* every sample is of `max_seq_len` length. (Unless the last batch only have one sample shorter than `max_seq_len`)

In [10]:
dls = db.dataloaders(df, shuffle_train=False, pad_first=False, bs=2,
                     dl_type=partial(TextDataloader,
                                     max_seq_len=7,
                                     agg_mode='lm',))
dls.show_batch(max_n=2)
print('x batch size:', dls.one_batch()[0].shape)

,text,text_
0,[CLS] monday left me broken [SEP] [CLS],monday left me broken [SEP] [CLS] tuesday
1,i was through with hoping [SEP] [CLS],was through with hoping [SEP] [CLS] wednesday


x batch size: torch.Size([2, 7])


# 4. Cache (developing)

In [0]:
def export_data(dataloaders, file):
  file = Path(file).resolve()
  if file.exists(): print('Append the data into the file. Note that this blends it with existing data in the file. ')
  for i, dl in enumerate(dataloaders):
    export_dataset(dl.dataset, file, i)

def export_dataset(dataset, file, dataset_idx=0):
  # h5 datasets will store variable 1d array of dtypes respectively
  dtypes = [ h5py.special_dtype(vlen=t.numpy().dtype)  for t in dataset[0] ] 
  
  with h5py.File(file, mode='a') as f:
    
    # f contains datasets (h5groups), first dataset will be train dataset
    if f'dataset_{dataset_idx}' in f.keys():
      ds = f[f'dataset_{dataset_idx}']
    else:
      ds = f.create_group(f'dataset_{dataset_idx}')

    # dataset contains data slices (h5groups), i th data slice is i th export of data
    # last character in the name of last group
    slice_idx = int(list(f.keys())[-1][-1]) + 1 if f.keys() else 0
    g = ds.create_group(f'slice_{slice_idx}')
    
    # data slice contain data elements (h5datasets), first data element will be Learner.xb[0]
    try:
      for i, (tl, dt) in enumerate(zip(dataset.tls, dtypes)):
        # data elment i contains samples[:][i] (1d numpy arrays)
        h5ds = g.create_dataset(name=str(i), shape=(len(dataset),), dtype=dt)
        h5ds[:] = tl[:]
    except Exception as e:
      del f[f'slice_{slice_idx}']
      raise e

In [0]:
class DatasetsByH5(FilteredBase):
  def __init__(self, h5_file, decode_funcs=None, load_to_memory=True):
    self.f = h5py.File(h5_file, mode='r', driver='core' if load_to_memory else None)
    self.decode_funcs = decode_funcs

  @property
  def n_subsets(self): return len(self.f)

  def subset(self, i):
    return DatasetByH5(self.f, list(self.f[f'dataset_{i}'].values()), decode_funcs=self.decode_funcs)
  
  def close(self): self.f.close()

def _slice_num_samples(data_slice): return len(data_slice['0']) # length of first data element

class DatasetByH5():
  def __init__(self, h5f, h5groups, decode_funcs=None):
    self.f = h5f
    self.slices = h5groups
    self.decode_funcs = decode_funcs if isinstance(decode_funcs, (list,tuple)) or decode_funcs is None else [decode_funcs]

  def __getitem__(self, i):
    for data_slice in self.slices:
      if i < _slice_num_samples(data_slice):
        # group is {'0':h5ds, '1':h5ds,...,'num_samples':int}  
        return tuple( torch.from_numpy(h5ds[i]) for h5ds in data_slice.values() ) 
      else:
        i -= _slice_num_samples(data_slice)

  def __len__(self):
    l = 0
    for data_slice in self.slices: l += _slice_num_samples(data_slice)
    return l

  def decode(self, sample, full=True): # full has not effect here
    assert self.decode_funcs is not None, 'No decode function passed.'
    return tuple( dec_fc(e) for e, dec_fc in zip(sample, self.decode_funcs))

  def close(self): self.f.close() # Note it may close other DatasetByH5 because of shared h5file

@delegates(FilteredBase.dataloaders)
def import_data(h5_file, load_to_memory=True, decode_funcs=None, **kwargs):
  dsets = DatasetsByH5(h5_file, load_to_memory=load_to_memory, decode_funcs=decode_funcs)
  try:
    return dsets.dataloaders(**kwargs)
  except Exception as e:
    dsets.close()
    raise e

In [0]:
export_data(dls, 'test_data.h5')

In [14]:
dlss = import_data('test_data.h5', bs=2,
                   dl_type=SortedDL,
                   dls_kwargs={'before_batch': partial(pad_input_chunk,
                                                       pad_idx=hf_tokenizer.pad_token_id,
                                                       pad_first=False)},
                   decode_funcs=[compose(hf_tokenizer.convert_ids_to_tokens, ' '.join ),]
                   )
dls.show_batch()

,text,text_
0,[CLS] monday left me broken [SEP] [CLS],monday left me broken [SEP] [CLS] tuesday
1,i was through with hoping [SEP] [CLS],was through with hoping [SEP] [CLS] wednesday


In [0]:
dlss.train_ds

In [15]:
dlss = import_data('test_data.h5', bs=2,
                   dl_type=partial(TextDataloader,
                                     max_seq_len=15,
                                     agg_mode='window',
                                     remove_heads=True,
                                     remove_tails=True,
                                     bos_idx_add=hf_tokenizer.cls_token_id,
                                     eos_idx_add=hf_tokenizer.sep_token_id),
                   dls_kwargs={'before_batch': partial(pad_input_chunk,
                                                       pad_idx=hf_tokenizer.pad_token_id,
                                                       pad_first=False)},
                   decode_funcs=[compose(hf_tokenizer.convert_ids_to_tokens, ' '.join ),]
                   )
dls.show_batch()

TypeError: ignored

In [0]:
dlss = import_data('test_data.h5', bs=2,
                   dl_type=partial(TextDataloader,
                                     max_seq_len=13,
                                     agg_mode='lines',
                                     remove_heads=True,
                                     bos_idx_add=hf_tokenizer.cls_token_id),
                   dls_kwargs={'before_batch': partial(pad_input_chunk,
                                                       pad_idx=hf_tokenizer.pad_token_id,
                                                       pad_first=False)},
                   decode_funcs=[compose(hf_tokenizer.convert_ids_to_tokens, ' '.join ),]
                   )
dls.show_batch()

In [0]:
dlss = import_data('test_data.h5', bs=2,
                   dl_type=partial(TextDataloader,
                                     max_seq_len=7,
                                     agg_mode='lm',)
                   dls_kwargs={'before_batch': partial(pad_input_chunk,
                                                       pad_idx=hf_tokenizer.pad_token_id,
                                                       pad_first=False)},
                   decode_funcs=[compose(hf_tokenizer.convert_ids_to_tokens, ' '.join ),]
                   )
dls.show_batch()